In [32]:
import pandas as pd
import numpy as np

In [33]:
data = pd.read_csv("IMDB Dataset.csv")
data.sample(10)

,review,sentiment
46351,They are hunted and starving. They are complet...,positive
2864,"Back in the day, I remembered seeing dumb Nint...",negative
33295,This is definitely a movie that will make you ...,positive
32006,"What a dreadful movie. The effects were poor, ...",negative
44931,The first Matrix movie was lush with incredibl...,negative
47241,(There Are Spoilers) Usual slasher film with t...,negative
30614,This is what happens when a franchise gets laz...,negative
35902,"As the story in my family goes, my dad, Milton...",positive
4237,"Well to answer one persons's question of ""why ...",negative
7529,Wow. This movie bored the pants off me when I ...,negative


In [34]:
data['label'] = data['sentiment'].replace(['positive', 'negative'],['1', '0'])

In [35]:
data.sample(10)

,review,sentiment,label
31280,"Look,I'm reading and reading this comments and...",negative,0
7845,Once when I was in college and we had an inter...,positive,1
20934,I like the good things in life as much as anyb...,positive,1
1956,I tuned into this thing one night on a cable c...,negative,0
43947,My watch came a little too late but am glad i ...,positive,1
42106,"My friend gives me these 3 huge boxes. ""They'r...",positive,1
7809,This is one of the movies having made signific...,positive,1
4879,Yeeee-Haa! <br /><br />I have seen it argued t...,negative,0
14024,<br /><br />I have seen this movie many times....,positive,1
16361,Although the plot of Cover Girl is very flimsy...,positive,1


In [36]:
data['label'].value_counts()

label
1    25000
0    25000
Name: count, dtype: int64

In [37]:
data.to_csv(r'C:\Users\ethan\OneDrive\Desktop\092023 SEM\NLP\\Assignment\Assignment 1\IMDB_Dataset_label.csv')

In [38]:
#Text Preprocessing for sentiment analysis
import string
import emoji
import re
import nltk
from textblob import TextBlob
from nltk.corpus import wordnet
from nltk.metrics import edit_distance
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize
import pandas as pd
stopwords = nltk.corpus.stopwords.words('english')

def preprocess(text):
    
    #1. Generating the list of words in the tweet (hastags and other punctuations removed)
    text_blob = TextBlob(text)
    text = ' '.join(text_blob.words)
    
    #2. clean the number 
    text = re.sub(r'[0-9]', '', text)
    
    #3. lower the text
    text = text.lower()
    
    #4. convert the emoji to text form
    text = emoji.demojize(text)
    
    #5. remove punctuation 
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    
    #6. tokenize the text
    text = word_tokenize(text)
    
    #7. remove empty token
    text = [t for t in text if len(t) > 0]
    
    #8. remove non-alphabetical token
    text = [t for t in text if t.isalpha()]
    
    #9. replace the negation token
    replacer  = AntonymReplacer()
    text = replacer.replace_negations(text)
    
    #10. remove the stopwords
    text = [i for i in text if i not in stopwords]
    
    #11. stem the text
    porter_stemmer = PorterStemmer()
    text = [porter_stemmer.stem(w) for w in text]
    
    return text

class AntonymReplacer(object):
    def replace(self, word, pos=None):
        antonyms = set()

        for syn in wordnet.synsets(word, pos=pos):
            for lemma in syn.lemmas():
                for antonym in lemma.antonyms():
                    antonyms.add(antonym.name())

        if len(antonyms) == 1:
            return antonyms.pop()
        else:
            return None

    def replace_negations(self, sent):
        i, l = 0, len(sent)
        words = []

        while i < l:
            word = sent[i]

            if word == 'not' and i+1 < l:
                ant = self.replace(sent[i+1])

                if ant:
                    words.append(ant)
                    i += 2
                    continue

            words.append(word)
            i += 1

        return words

In [39]:
print(data['review'].iloc[12])
print(preprocess(data['review'].iloc[12]))

So im not a big fan of Boll's work but then again not many are. I enjoyed his movie Postal (maybe im the only one). Boll apparently bought the rights to use Far Cry long ago even before the game itself was even finsished. <br /><br />People who have enjoyed killing mercs and infiltrating secret research labs located on a tropical island should be warned, that this is not Far Cry... This is something Mr Boll have schemed together along with his legion of schmucks.. Feeling loneley on the set Mr Boll invites three of his countrymen to play with. These players go by the names of Til Schweiger, Udo Kier and Ralf Moeller.<br /><br />Three names that actually have made them selfs pretty big in the movie biz. So the tale goes like this, Jack Carver played by Til Schweiger (yes Carver is German all hail the bratwurst eating dudes!!) However I find that Tils acting in this movie is pretty badass.. People have complained about how he's not really staying true to the whole Carver agenda but we on

In [40]:
# Train the Countvectorizer

import pickle

from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(analyzer=preprocess)
bow_vector=vectorizer.fit_transform(data['review'])



In [41]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
SEED = 4000

X_train, X_test, y_train, y_test = train_test_split(bow_vector, 
                                            data.label, test_size=0.2, random_state=SEED)

print('X_train shape: {}'.format(X_train.shape))
print('X_test shape: {}'.format(X_test.shape))
print('y_train shape: {}'.format(y_train.shape))
print('y_test shape: {}'.format(y_test.shape))

X_train shape: (40000, 101981)
X_test shape: (10000, 101981)
y_train shape: (40000,)
y_test shape: (10000,)


In [42]:
from sklearn.svm import SVC


clf_svm = SVC()

clf_names = ['SVM']

clf_types = [clf_svm]

for (i, clf) in enumerate(clf_types):
    clf.fit(X_train, y_train.values.ravel())
    print('Result of {}\n'.format(clf_names[i]))
    predictions = clf.predict(X_test)
    print(classification_report(predictions, y_test))
    print('\n')
    print('Confusion matrix: \n', confusion_matrix(predictions, y_test))
    print('\n')
    print('Accuracy score: ', accuracy_score(predictions, y_test))
    print('\n\n\n')

Result of SVM

              precision    recall  f1-score   support

           0       0.85      0.90      0.88      4761
           1       0.90      0.86      0.88      5239

    accuracy                           0.88     10000
   macro avg       0.88      0.88      0.88     10000
weighted avg       0.88      0.88      0.88     10000



Confusion matrix: 
 [[4280  481]
 [ 726 4513]]


Accuracy score:  0.8793






In [43]:
#save vectorizer
vectorizer_file = r'C:\Users\ethan\OneDrive\Desktop\092023 SEM\NLP\Assignment\Assignment 1\ModelFolder/SVM_CountVectorizer.sav';
pickle.dump(clf_svm, open(vectorizer_file, 'wb'))